In [1]:
import pandas as pd
import numpy as np
import scipy.signal as signal
import os
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import rcParams
text_color = 'black'
rcParams['text.color'] = text_color
rcParams['axes.labelcolor'] = text_color
rcParams['xtick.color'] = text_color
rcParams['ytick.color'] = text_color
plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=20)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=26)  # fontsize of the figure title

In [ ]:
def extract_results(path, pop):
       
    events = pd.read_csv(path+'events_pop_'+pop+'.csv')
    frequency = pd.read_csv(path+'frequency.csv')
    stimuli = pd.read_csv(path+'stimuli.csv')

    evs = events['ID neuron pop_'+pop].to_numpy()
    ts = events['event time pop_'+pop].to_numpy()
    t = frequency['time'].to_numpy()
    activity = frequency['activity (Hz) pop_'+pop].to_numpy()
    if pop=='inh':
        stimulus = False
        sum_stimulus = False
    else:
        stimulus = stimuli['stimulus pop '+pop].to_numpy()
        sum_stimulus = stimuli['integral stim pop '+pop].to_numpy()


    return evs, ts, t, activity, stimulus, sum_stimulus

## Figure 1 
#### Raster plot and Activity plot
Comparison of the standard behaviour of the network at 0.0%, 12.8% and 51.2% stimulus coherence

In [ ]:
show=True

sim_parameters = pd.read_csv('simulation_parameters.csv', index_col=0)
sim_type = 'standard'
start_stim = sim_parameters[sim_type]['start_stim']
end_stim = sim_parameters[sim_type]['end_stim']
results_dir = 'results/'+sim_type
figure_dir = 'figures'

fig1,axes = plt.subplots(6, 2, figsize=(6,10),sharex=True,constrained_layout=True)

mult_coherence = [0.0,-0.128,-0.512]

trial = [158,199,6]

for i, coherence in enumerate(mult_coherence):
    j = trial[i]
    path = results_dir+'/c'+str(coherence) +'/trial_'+ str(j)+'/'
    evsA, tsA, t, A_N_A, stimulus_A, sum_stimulus_A = extract_results(path, 'A')
    evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
    evsIn, tsIn, t, I_N_I, stimulus_A, sum_stimulus_A = extract_results(path, 'inh')
    I_N_I=signal.medfilt(I_N_I,55) 

    axes[2*i][0].scatter(tsA, evsA,marker = '|', linewidths = 0.8, color='red', label ='pop A')
    axes[2*i][0].set_ylabel("neuron #")
    axes[2*i][0].vlines(start_stim, 0, 800, color='grey')
    axes[2*i][0].vlines(end_stim, 0, 800, color='grey')
    axes[2*i][0].set_yticks([]) 
    axes[2*i+1][0].plot(t, A_N_A, color='red', label ='pop A')
    axes[2*i+1][0].fill_between(t, A_N_A, color='red')
    axes[2*i+1][0].vlines(start_stim, 0, 60, color='grey')
    axes[2*i+1][0].vlines(end_stim, 0, 60, color='grey')
    axes[2*i+1][0].set_ylabel("A(t) [Hz]")
    axes[2*i+1][0].set_ylim(0,60)

    axes[2*i][1].scatter(tsB, evsB,marker= '|',  linewidths = 0.8,color='blue', label ='pop B')
    axes[2*i][1].vlines(start_stim, 800, 1600, color='grey')
    axes[2*i][1].vlines(end_stim, 800, 1600, color='grey')
    axes[2*i][1].set_yticks([]) 
    axes[2*i+1][1].plot(t, B_N_B, color='blue', label ='pop B')
    axes[2*i+1][1].fill_between(t, B_N_B, color='blue')
    axes[2*i+1][1].vlines(start_stim, 0, 60, color='grey')
    axes[2*i+1][1].vlines(end_stim, 0, 60, color='grey')
    axes[2*i+1][1].set_ylim(0,60)
    axes[2*i+1][1].set_yticks([])

axes[5][0].set_xlabel("t [ms]")
axes[5][1].set_xlabel("t [ms]")

fig1.savefig(figure_dir+'/Figure1.png', bbox_inches='tight')

if show:
    plt.show()


## Figure 2
### Activity of inhibitory cell
2a. Comparison of the inhibitory activty at 0.0%, 12.8% and 51.2% stimulus coherence\
2b. Decision space at 0.0%, 12.8% and 51.2% stimulus coherence

In [ ]:
show=True

sim_parameters = pd.read_csv('simulation_parameters.csv', index_col=0)
sim_type = 'standard'
results_dir = 'results/'+sim_type
figure_dir = 'figures'

fig2,axes = plt.subplots(2, 1,  figsize=(6,10),constrained_layout=True)

mult_coherence = [0.0,-0.128,-0.512]
color_A= ['tomato','r','darkred']
color_inh = ['darkgrey','dimgrey','k']

trial = [158,199,6]

axes[1].plot([0,40],[0,40], color='grey')
for i, coherence in enumerate(mult_coherence):
    j = trial[i]
    path = results_dir+'c'+str(coherence) +'/trial_'+ str(j)+'/'
    evsA, tsA, t, A_N_A, stimulus_A, sum_stimulus_A = extract_results(path, 'A')
    evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
    evsIn, tsIn, t, I_N_I, stimulus_A, sum_stimulus_A = extract_results(path, 'inh')
    I_N_I=signal.medfilt(I_N_I,55) 
    axes[0].plot(t[:120], I_N_I[:120], color=color_inh[i],alpha= 0.9, label ='c '+str(coherence*100)+'%')
    axes[1].plot(A_N_A,B_N_B,color=color_A[i], alpha= 0.9, label ='c '+str(coherence*100)+'%')

axes[0].set_ylabel("A(t) [Hz]")
axes[0].set_xlabel("t [ms]")
axes[0].set_aspect(10)
axes[0].set_ylim(0,60)
axes[1].set_xlim(-0.1,40)
axes[1].set_ylim(-0.1,40)
axes[1].set_xlabel("Firing rate pop A (Hz)")
axes[1].set_ylabel("Firing rate pop B (Hz)")    

fig2.savefig(figure_dir+'/Figure2.png', bbox_inches='tight')

if show:
    plt.show()

## Figure 3

#### Effect of recurrent excitatory connection and NMDA slow dynamic
3a. Comparison of the activity at 0.0% and 51.2% when the recurrent weight are decreased\
3b. Comparison of the activity at 0.0% and 51.2% when the tua_syn of NMDA receptor is equal to 2 ms (as AMPA)

In [ ]:
show=True

sim_parameters = pd.read_csv('simulation_parameters.csv', index_col=0)
figure_dir = 'figures'

fig3, axes = plt.subplots(2, 4, sharey=True,sharex=True,  figsize=(10,10))

mult_coherence = [0.0, 0.512]

for i, coherence in enumerate(mult_coherence):

    path_standard = 'results/standard/c'+str(coherence)
    trial_winner = pd.read_csv(path_standard+'/trial_winner.csv')
    trial_B_win = trial_winner['trial'][trial_winner['winner'=='B']].to_numpy()
    A_N_A_mean =[]
    B_N_B_mean =[]
    for j in trial_B_win:
        path = path_standard + '/trial_'+str(j)+'/'
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        B_N_B_mean.append(B_N_B)
        evsA, tsA, t, A_N_A, stimulus_A, sum_stimulus_A = extract_results(path, 'A')
        A_N_A_mean.append(A_N_A)     
    B_N_B_mean = np.mean(B_N_B_mean,axis=0)
    B_N_B_smooth=signal.medfilt(B_N_B_mean,55) 
    A_N_A_mean = np.mean(A_N_A_mean,axis=0)
    A_N_A_smooth=signal.medfilt(A_N_A_mean,55) 

    path_decrease = 'results/decrease/c'+str(coherence)
    A_N_A_mean_decrease =[]
    B_N_B_mean_decrease =[]
    for path in os.listdir(path_decrease+'/'):
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        B_N_B_mean_decrease.append(B_N_B)
        evsA, tsA, t, A_N_A, stimulus_A, sum_stimulus_A = extract_results(path, 'A')
        A_N_A_mean_decrease.append(A_N_A)
    B_N_B_mean_decrease = np.mean(B_N_B_mean_decrease,axis=0)
    B_N_B_smooth_decrease=signal.medfilt(B_N_B_mean_decrease,55)
    A_N_A_mean_decrease = np.mean(A_N_A_mean_decrease,axis=0)
    A_N_A_smooth_decrease=signal.medfilt(A_N_A_mean_decrease,55) 

    path_no_NMDA = 'results/no_NMDA/c'+str(coherence)
    A_N_A_mean_no_NMDA =[]
    B_N_B_mean_no_NMDA =[]
    for path in os.listdir(path_no_NMDA+'/'):
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        B_N_B_mean_no_NMDA.append(B_N_B)
        evsA, tsA, t, A_N_A, stimulus_A, sum_stimulus_A = extract_results(path, 'A')
        A_N_A_mean_no_NMDA.append(A_N_A)
    B_N_B_mean_no_NMDA = np.mean(B_N_B_mean_no_NMDA,axis=0)
    B_N_B_smooth_no_NMDA=signal.medfilt(B_N_B_mean_no_NMDA,55)
    A_N_A_mean_no_NMDA = np.mean(A_N_A_mean_no_NMDA,axis=0)
    A_N_A_smooth_no_NMDA=signal.medfilt(A_N_A_mean_no_NMDA,55) 

    axes[i][0].plot(t,B_N_B_mean_decrease,'blue')
    axes[i][0].plot(t,B_N_B_smooth_decrease,'green')
    axes[i][0].plot(t,B_N_B_mean,'blue',alpha=0.6)
    axes[i][0].plot(t,B_N_B_smooth,'green')
    axes[i][0].set_ylim(0,60)
    axes[i][0].set_ylabel("A(t) [Hz]")
    axes[i][0].vlines(start_stim, 0, 60, color='grey')
    axes[i][0].vlines(end_stim, 0, 60, color='grey')
    axes[i][1].plot(t,A_N_A_mean_decrease,'red')
    axes[i][1].plot(t,A_N_A_smooth_decrease,'green')
    axes[i][1].plot(t,A_N_A_mean,'red',alpha=0.6)
    axes[i][1].plot(t,A_N_A_smooth,'green')
    axes[i][1].set_ylim(0,60)
    axes[i][1].vlines(start_stim, 0, 60, color='grey')
    axes[i][1].vlines(end_stim, 0, 60, color='grey')

    axes[i][2].plot(t,B_N_B_mean_decrease,'blue')
    axes[i][2].plot(t,B_N_B_smooth_decrease,'green')
    axes[i][2].plot(t,B_N_B_mean,'blue',alpha=0.6)
    axes[i][2].plot(t,B_N_B_smooth,'green')
    axes[i][2].set_ylim(0,60)
    axes[i][2].set_ylabel("A(t) [Hz]")
    axes[i][2].vlines(start_stim, 0, 60, color='grey')
    axes[i][2].vlines(end_stim, 0, 60, color='grey')
    axes[i][3].plot(t,A_N_A_mean_decrease,'red')
    axes[i][3].plot(t,A_N_A_smooth_decrease,'green')
    axes[i][3].plot(t,A_N_A_mean,'red',alpha=0.6)
    axes[i][3].plot(t,A_N_A_smooth,'green')
    axes[i][3].set_ylim(0,60)
    axes[i][3].vlines(start_stim, 0, 60, color='grey')
    axes[i][3].vlines(end_stim, 0, 60, color='grey')
        
    axes[1][0].set_xlabel("t [ms]")
    axes[1][1].set_xlabel("t [ms]")
    axes[1][2].set_xlabel("t [ms]")
    axes[1][3].set_xlabel("t [ms]")

fig3.savefig('figures/Figure3.png' , bbox_inches='tight')

if show:
    plt.show()

## Figure 4
Coin toss symmetry: behaviour of the network at 0.0% coherence

In [ ]:
show=True

sim_parameters = pd.read_csv('simulation_parameters.csv', index_col=0)
sim_type = 'standard'
start_stim = sim_parameters[sim_type]['start_stim']
end_stim = sim_parameters[sim_type]['end_stim']
simtime = sim_parameters[sim_type]['simtime']

figure_dir = 'figures'
results_dir = 'results/'+sim_type

fig4, axes = plt.subplots(4, 2, sharex=True, figsize=(16,9))

coherence = 0.0
trials = [158,146]

for j in trials:
    path = results_dir+'/c'+str(coherence)+ '/trial_'+ str(j)+'/'
    evsA, tsA, t, A_N_A, stimulus_A, sum_stimulus_A = extract_results(path, 'A')
    evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')

    axes[0][i].scatter(tsA, evsA,marker = '|', linewidths = 0.8, color='red', label ='pop A')
    axes[0][i].scatter(tsB, evsB,marker = '|', linewidths = 0.8, color='blue', label ='pop B')
    axes[0][i].vlines(start_stim, 0, 1600, color='grey')
    axes[0][i].vlines(end_stim, 0, 1600, color='grey')
    axes[0][i].set_ylabel("neuron #")
    axes[0][i].set_title("Raster Plot ", fontsize=10)

    axes[1][i].plot(t, A_N_A, color='red', label ='pop A')
    axes[1][i].plot(t, B_N_B, color='blue', label ='pop B')
    axes[1][i].vlines(start_stim, 0, 40, color='grey')
    axes[1][i].vlines(end_stim, 0, 40, color='grey')
    axes[1][i].set_ylabel("A(t) [Hz]")
    axes[1][i].set_title("Activity", fontsize=10)

    axes[2][i].plot(np.arange(0., simtime),stimulus_A, 'red', label='stimulus on A')
    axes[2][i].plot(np.arange(0., simtime),stimulus_B, 'blue', label='stimulus on B')
    axes[2][i].set_ylabel("stimulus [Hz]")
    axes[2][i].set_title("Stochastic inputs", fontsize=10)

    axes[3][i].plot(np.arange(0., simtime),sum_stimulus_A, 'red', label='sum_stimulus on A')
    axes[3][i].plot(np.arange(0., simtime),sum_stimulus_B, 'blue', label='sum_stimulus on B')
    axes[3][i].set_title("Time integral of inputs", fontsize=10)
    axes[3][i].set_yticks([])
    plt.xlabel("t [ms]")

fig4.savefig('figures/Figure4.png' , bbox_inches='tight')

if show:
    plt.show()

## Figure 5
Performance of the network\
5a. Percentage of correct choice\
5b. Reaction time on activity plot\
5c. Counts on reaction times\
5d. non mi ricordo

In [ ]:
show=True
sim_type = 'standard'
start_stim = sim_parameters[sim_type]['start_stim']
end_stim = sim_parameters[sim_type]['end_stim']
simtime = sim_parameters[sim_type]['simtime']
dt_rec = sim_parameters[sim_type]['dt_rec']

figure_dir = 'figures'
results_dir = 'results/'+sim_type

def weib(c,alpha,beta):
    return 1-0.5*np.exp(-(c/alpha)**beta)

fig5, axes = plt.subplots(3, 3, sharey=True,sharex=True,  figsize=(10,10))

#########################################################################################
n_trial = 1000
winner = pd.read_csv(results_dir+'/1000_winners.csv')
percentage_B_win = 100*(winner['pop B win'].to_numpy()[:6])/n_trial
percentage_A_win = 100*(winner['pop A win'].to_numpy()[8:])/n_trial

'''
percentage_B_win=[]
mult_coherence = [0.032,0.064,0.128,0.256,0.512]
for i, coherence in enumerate(mult_coherence):
    path_standard = results_dir+'/c'+str(coherence)
    trial_winner = pd.read_csv(path_standard+'/trial_winner.csv')
    trial_B_win = trial_winner['winner'=='B'].to_numpy()
    percentage_B_win.append((len(trial_B_win)/n_trial)*100)

percentage_A_win=[]
mult_coherence = [-0.032,-0.064,-0.128,-0.256,-0.512]
for i, coherence in enumerate(mult_coherence):
    path_standard = results_dir+'/c'+str(coherence)
    trial_winner = pd.read_csv(path_standard+'/trial_winner.csv')
    trial_A_win = trial_winner['winner'=='A'].to_numpy()
    percentage_A_win.append((len(trial_A_win)/n_trial)*100)
    '''

coherence_level = np.array([3.2,6.4,12.8,25.6,51.2,100.])
c=np.linspace(3.2,100.,100)
fit_weibul=weib(c,9.2,1.8)
point_weibul=weib(coherence_level,9.2,1.8)
axes[0][0].semilogx(coherence_level,percentage_A_win,'s-', color='red')
axes[0][0].semilogx(coherence_level,percentage_B_win,'o-', color='blue')
axes[0][0].semilogx(c,fit_weibul*100,color='black')
axes[0][0].semilogx(coherence_level,point_weibul*100,'*',color='black')
axes[0][0].set_xlim(2*1e0, 2*1e2)
axes[0][0].set_xlabel('Coherence level %')
axes[0][0].set_ylabel('%\ of correct choice')
#########################################################################################
coherence_level = [0.032, 0.064, 0.128]
n_trial = 200
for i,coherence in enumerate(coherence_level):

    path_standard = results_dir+'/c'+str(coherence)
    trial_winner = pd.read_csv(path_standard+'/trial_winner.csv')
    trial_B_win = trial_winner['winner'=='B'].to_numpy()
    trial_A_win = trial_winner['winner'=='A'].to_numpy()

    mean_activity_wB_sB = []
    for j in range(trial_B_win): 
        path = path_standard+'/trial_'+ str(j) + '/'
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        mean_activity_wB_sB.append(B_N_B)

    mean_activity_wA_sB = []
    for j in range(trial_A_win): 
        path = path_standard+'/trial_'+ str(j) + '/'
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        mean_activity_wA_sB.append(B_N_B)
        
    mean_activity_wB_sB = np.mean(mean_activity_wB_sB,axis=0)
    mean_activity_wA_sB = np.mean(mean_activity_wA_sB,axis=0)
    mean_activity_wB_sB = signal.medfilt(mean_activity_wB_sB,35)
    mean_activity_wA_sB = signal.medfilt(mean_activity_wA_sB,35)
    
    axes[i][2].plot(t[0:120],mean_activity_wB_sB[0:120],'black')
    axes[i][2].plot(t[0:120],mean_activity_wA_sB[0:120],'orange','--',alpha=0.6)

coherence_level = [-0.032, -0.064, -0.128]
for i,coherence in enumerate(coherence_level):

    path_standard = 'results/standard/c'+str(coherence)
    trial_winner = pd.read_csv(path_standard+'/trial_winner.csv')
    trial_B_win = trial_winner['winner'=='B'].to_numpy()
    trial_A_win = trial_winner['winner'=='A'].to_numpy()

    mean_activity_wB_sA = []
    for j in range(trial_B_win): 
        path = path_standard+'/trial_'+ str(j) + '/'
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        mean_activity_wB_sA.append(B_N_B)


    mean_activity_wA_sA = []
    for j in range(trial_A_win): 
        path = path_standard+'/trial_'+ str(j) + '/'
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        mean_activity_wA_sA.append(B_N_B)
    
    mean_activity_wB_sA = np.mean(mean_activity_wB_sA,axis=0)
    mean_activity_wA_sA = np.mean(mean_activity_wA_sA,axis=0)
    mean_activity_wB_sA = signal.medfilt(mean_activity_wB_sA,35)
    mean_activity_wA_sA = signal.medfilt(mean_activity_wA_sA,35)
    
    axes[i][2].plot(t[0:120],mean_activity_wB_sA[0:120],'black', '--',alpha=0.6)
    axes[i][2].plot(t[0:120],mean_activity_wA_sA[0:120],'orange')    
    axes[i][2].set_ylim(0,40)

#########################################################################################
mult_coherence = [0.0,0.512]
thr_activity = 15
ind_start_stim = int(start_stim/dt_rec)
ind_end_stim = int(end_stim/dt_rec)
for i, coherence in enumerate(mult_coherence):
    path_standard = 'results/standard/c'+str(coherence)
    trial_winner = pd.read_csv(path_standard+'/trial_winner.csv')
    trial_B_win = trial_winner['winner'=='B'].to_numpy()
    B_N_B_mean = []
    decision_time = []
    for j in trial_B_win:
        path = results_dir+'/trial_'+ str(j) + '/'
        evsB, tsB, t, B_N_B, stimulus_B, sum_stimulus_B = extract_results(path, 'B')
        B_N_B_mean.append(B_N_B)            
        B_N_B = signal.medfilt(B_N_B,35)  
        axes[1][i].plot(t[ind_start_stim:ind_end_stim], B_N_B[ind_start_stim:ind_end_stim], color='black', label ='pop B')
        axes[1][i].hlines(15, start_stim, end_stim, 'grey')
        axes[1][i].set_ylim(0,30)
        axes[1][i].set_xlabel("Time [ms]")
        if len(t[B_N_B >= thr_activity])!=0:
            decision_time.append(t[B_N_B >= thr_activity][0])  

    axes[2][0].hist(decision_time, color = [0.5-i*0.3,0.4-i*0.3,0.5-i*0.3], linewidth = 2)

    B_N_B_mean = np.mean(B_N_B_mean,axis=0)
    B_N_B_mean = signal.medfilt(B_N_B_mean,35)
    axes[i].plot(t[ind_start_stim:ind_end_stim], B_N_B_mean[ind_start_stim:ind_end_stim], color='green', label ='pop B')

axes[1][0].set_ylabel("A(t) [Hz]")
axes[2][0].set_xlabel('Decision time [ms]')
axes[2][0].set_ylabel('Counts #')

fig5.savefig('figures/Figure5.png' , bbox_inches='tight')

if show:
    plt.show()

## Figure 6

## Figure 7